In [1]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.linear_model import LogisticRegression


In [2]:
#THIS IS IT

In [3]:
## Loading the data
train_df = pd.read_csv('/app/digicow/data/Train.csv')
test_df = pd.read_csv('/app/digicow/data/Test.csv')
#y_true = pd.read_csv('reference.csv')['Target_AUC']

In [ ]:
train_df_ = pd.read_csv('/app/digicow/data/Train_.csv')
test_df_ = pd.read_csv('/app/digicow/data/Test_.csv')
#y_true_ = pd.read_csv('reference_.csv')['Target_AUC']

In [32]:
for c in [col for col in train_df.columns if col.endswith('date')]:
  if c in train_df.columns:
    train_df[c] = pd.to_datetime(train_df[c], errors='coerce')


for c in [col for col in train_df_.columns if col.endswith('date')]:
  if c in train_df_.columns:
    train_df_[c] = pd.to_datetime(train_df_[c], errors='coerce')


In [33]:
target_col = 'adopted_within_07_days'
train_df[target_col] = train_df[target_col].astype(int)

target_col = 'adopted_within_07_days'
train_df_[target_col] = train_df_[target_col].astype(int)

In [34]:
for c in [col for col in test_df.columns if col.endswith('date')]:
  if c in test_df.columns:
    test_df[c] = pd.to_datetime(test_df[c], errors='coerce')


for c in [col for col in test_df_.columns if col.endswith('date')]:
  if c in test_df_.columns:
    test_df_[c] = pd.to_datetime(test_df_[c], errors='coerce')

In [35]:
split_summary = pd.DataFrame({
    "set": ["train"],
    "rows": [len(train_df)],
    "positives": [train_df[target_col].sum()],
})
split_summary["pos_rate"] = split_summary["positives"] / split_summary["rows"]

split_summary

,set,rows,positives,pos_rate
0,train,5525,617,0.111674


In [36]:
split_summary_ = pd.DataFrame({
    "set": ["train"],
    "rows": [len(train_df_)],
    "positives": [train_df_[target_col].sum()],
})
split_summary_["pos_rate"] = split_summary_["positives"] / split_summary_["rows"]

split_summary_

,set,rows,positives,pos_rate
0,train,27381,617,0.022534


In [37]:
## Basic model with some selected features
base_features = [
    "gender",
    "registration",
    "age",
    "trainer",
    "belong_to_cooperative",
    "county",
    "subcounty",
    "ward",
]

In [38]:
feature_cols = base_features

X_train = train_df[feature_cols]
y_train = train_df[target_col]

X_test  = test_df[feature_cols]


X_train_ = train_df_[feature_cols]
y_train_ = train_df_[target_col]

X_test_  = test_df_[feature_cols]

## Make the prediction


In [39]:
preprocess = ColumnTransformer(
    transformers=[
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), feature_cols)
    ]
)

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=3000, class_weight="balanced"))
])

model.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

In [40]:
preprocess_ = ColumnTransformer(
    transformers=[
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), feature_cols)
    ]
)

model_ = Pipeline(steps=[
    ("preprocess", preprocess_),
    ("clf", LogisticRegression(max_iter=3000, class_weight="balanced"))
])

model_.fit(X_train_, y_train_)
y_pred_prob_ = model.predict_proba(X_test_)[:, 1]
y_pred_ = model_.predict(X_test_)

In [41]:
y_pred

array([1, 0, 1, ..., 0, 0, 0])

In [42]:
## Evaluating some metrics
# ----------------------------
#  Metrics: PR AUC + ROC AUC + Recall@K
# ----------------------------
def recall_at_k(y_true: pd.Series, y_scores: np.ndarray, k_frac: float) -> float:
    k = int(np.ceil(len(y_true) * k_frac))
    order = np.argsort(-y_scores)  # descending
    topk = order[:k]
    return float(y_true.iloc[topk].sum() / y_true.sum()) if y_true.sum() > 0 else np.nan

In [43]:
pr_auc = average_precision_score(y_true, y_pred_prob)
roc_auc = roc_auc_score(y_true, y_pred_prob)
recall_5 = recall_at_k(y_true, y_pred_prob, 0.05)
recall_10 = recall_at_k(y_true, y_pred_prob, 0.10)
recall_20 = recall_at_k(y_true, y_pred_prob, 0.20)

# Naive baselines on this test set
prevalence_test = y_true.mean()
naive_pr_auc = prevalence_test

results = pd.DataFrame({
    "metric": [
        "PR AUC (Average Precision)",
        "ROC AUC",
        "Recall@5%",
        "Recall@10%",
        "Recall@20%",
        "Naive PR AUC (prevalence)",
    ],
    "value": [pr_auc, roc_auc, recall_5, recall_10, recall_20, naive_pr_auc]
})


print("\nSplit summary:")
display(split_summary)
print("\nResults:")
display(results)


Split summary:


,set,rows,positives,pos_rate
0,train,5525,617,0.111674



Results:


,metric,value
0,PR AUC (Average Precision),0.441386
1,ROC AUC,0.860209
2,Recall@5%,0.266917
3,Recall@10%,0.368421
4,Recall@20%,0.597744
5,Naive PR AUC (prevalence),0.111812


In [44]:
y_pred_prob

array([0.66407633, 0.32957527, 0.59670526, ..., 0.00681731, 0.04802049,
       0.06112566])

In [45]:
y_pred

array([1, 0, 1, ..., 0, 0, 0])

In [46]:
pr_auc_ = average_precision_score(y_true_, y_pred_prob_)
roc_auc_ = roc_auc_score(y_true_, y_pred_prob_)
recall_5_ = recall_at_k(y_true_, y_pred_prob_, 0.05)
recall_10_ = recall_at_k(y_true_, y_pred_prob_, 0.10)
recall_20_ = recall_at_k(y_true_, y_pred_prob_, 0.20)

# Naive baselines on this test set
prevalence_test_ = y_true_.mean()
naive_pr_auc_ = prevalence_test_

results_ = pd.DataFrame({
    "metric": [
        "PR AUC (Average Precision)",
        "ROC AUC",
        "Recall@5%",
        "Recall@10%",
        "Recall@20%",
        "Naive PR AUC (prevalence)",
    ],
    "value": [pr_auc_, roc_auc_, recall_5_, recall_10_, recall_20_, naive_pr_auc_]
})


print("\nSplit summary:")
display(split_summary_)
print("\nResults:")
display(results_)


Split summary:


,set,rows,positives,pos_rate
0,train,27381,617,0.022534



Results:


,metric,value
0,PR AUC (Average Precision),0.165191
1,ROC AUC,0.868006
2,Recall@5%,0.327068
3,Recall@10%,0.484962
4,Recall@20%,0.695489
5,Naive PR AUC (prevalence),0.022646


In [47]:
## Make submission file
ss = pd.read_csv('SampleSubmission.csv')
ss['Target_LogLoss'] = y_pred_prob
ss['Target_AUC'] = y_pred_prob
ss.to_csv('BenchmarkSub.csv', index=False)

ss_ = pd.read_csv('SampleSubmission_.csv')
ss_['Target_LogLoss'] = y_pred_prob_
ss_['Target_AUC'] = y_pred_prob_
ss_.to_csv('BenchmarkSub_.csv', index=False)

In [48]:
ss.head()

,ID,Target_AUC,Target_LogLoss
0,ID_VR75BG,0.664076,0.664076
1,ID_D2J0WZ,0.329575,0.329575
2,ID_CWLNAK,0.596705,0.596705
3,ID_9BU2DP,0.946992,0.946992
4,ID_QTX80L,0.360873,0.360873


In [49]:
ss_.head()

,ID,Target_AUC,Target_LogLoss
0,ID_1MNN3I,0.619965,0.619965
1,ID_XOT5G5,0.219667,0.219667
2,ID_OZN7P3,0.783224,0.783224
3,ID_LT5R6J,0.188326,0.188326
4,ID_T35OKD,0.014446,0.014446


In [50]:
ss

,ID,Target_AUC,Target_LogLoss
0,ID_VR75BG,0.664076,0.664076
1,ID_D2J0WZ,0.329575,0.329575
2,ID_CWLNAK,0.596705,0.596705
3,ID_9BU2DP,0.946992,0.946992
4,ID_QTX80L,0.360873,0.360873
...,...,...,...
2374,ID_KM5D53,0.192079,0.192079
2375,ID_WNTVRD,0.016103,0.016103
2376,ID_7N936W,0.006817,0.006817
2377,ID_2PJRH3,0.048020,0.048020


In [51]:
ss_

,ID,Target_AUC,Target_LogLoss
0,ID_1MNN3I,0.619965,0.619965
1,ID_XOT5G5,0.219667,0.219667
2,ID_OZN7P3,0.783224,0.783224
3,ID_LT5R6J,0.188326,0.188326
4,ID_T35OKD,0.014446,0.014446
...,...,...,...
11741,ID_NDOMA0,0.006505,0.006505
11742,ID_LN2XMC,0.043849,0.043849
11743,ID_EOIZLZ,0.004697,0.004697
11744,ID_GENFFO,0.006817,0.006817


In [52]:
from sklearn.metrics import log_loss

In [53]:
## Compute the log-loss
loss = log_loss(y_true, y_pred_prob)
print("LogLoss:", loss)

loss_ = log_loss(y_true_, y_pred_prob_)
print("LogLoss:", loss_)

LogLoss: 0.4692600295166461
LogLoss: 0.4678933473746918


In [54]:
train_df.columns

Index(['ID', 'gender', 'age', 'registration', 'belong_to_cooperative',
       'county', 'subcounty', 'ward', 'trainer', 'first_training_date',
       'topics_list', 'num_trainings_30d', 'num_trainings_60d',
       'num_total_trainings', 'num_repeat_trainings',
       'days_to_second_training', 'num_unique_trainers',
       'adopted_within_07_days', 'has_second_training'],
      dtype='object')

In [55]:
train_df_.columns

Index(['ID', 'gender', 'age', 'registration', 'belong_to_cooperative',
       'county', 'subcounty', 'ward', 'trainer', 'first_training_date',
       'topics_list', 'num_trainings_30d', 'num_trainings_60d',
       'num_total_trainings', 'num_repeat_trainings',
       'days_to_second_training', 'num_unique_trainers',
       'adopted_within_07_days', 'adopted_within_30_days',
       'adopted_within_60_days', 'adopted_within_90_days',
       'adopted_within_120_days', 'has_second_training'],
      dtype='object')